In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from torchvision.datasets import MNIST, CIFAR10

from utils import training, evaluation
from models import bp_model, cnn_model

# 全局配置
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CRITERION = nn.CrossEntropyLoss()

# 数据集加载与预处理
def load_datasets():
    transform = transforms.Compose([transforms.ToTensor()])
    train_ds_map = {}
    test_ds_map  = {}
    train_ds_map['mnist'], test_ds_map['mnist']   = MNIST('./data', True, download=True,  transform=transform), MNIST('./data', False, download=True, transform=transform)
    train_ds_map['cifar10'], test_ds_map['cifar10'] = CIFAR10('./data', True, download=True, transform=transform), CIFAR10('./data', False, download=True, transform=transform)
    return train_ds_map, test_ds_map

train_ds_map, test_ds_map = load_datasets()
input_shape_map = { ds: train_ds_map[ds][0][0].shape for ds in train_ds_map }

# 统一测试集 DataLoader
def get_test_loader(dataset, batch_size=64):
    return DataLoader(test_ds_map[dataset], batch_size=batch_size, shuffle=False)

test_loader_map = { ds: get_test_loader(ds) for ds in test_ds_map }

# 交叉验证划分
def get_folds(dataset, n_splits=5, random_state=42):
    from sklearn.model_selection import KFold
    indices = np.arange(len(train_ds_map[dataset]))
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    return list(kf.split(indices))

# 运行单次实验
def run_experiment(dataset, model_type, epochs=10, batch_size=32, **model_kwargs):
    test_accs = []
    folds = get_folds(dataset)
    for tr_idx, va_idx in folds:
        tr_loader = DataLoader(Subset(train_ds_map[dataset], tr_idx), batch_size=batch_size, shuffle=True)
        va_loader = DataLoader(Subset(train_ds_map[dataset], va_idx), batch_size=batch_size, shuffle=False)
        # 模型构建
        model_fn = bp_model.create_bp_model if model_type=='bp' else cnn_model.create_cnn_model
        model = model_fn(input_shape_map[dataset], num_classes=10).to(DEVICE)
        optimizer = optim.Adam(model.parameters(), lr=model_kwargs.get('lr', 1e-3))
        # 训练
        training.train_model(model, tr_loader, va_loader, epochs, DEVICE, CRITERION, optimizer)
        # 测试
        _, acc = evaluation.evaluate_model(model, test_loader_map[dataset], DEVICE, CRITERION)
        test_accs.append(acc)
    return np.array(test_accs)






Files already downloaded and verified
Files already downloaded and verified


In [3]:

# 4.1 对比实验
models  = ['bp', 'cnn']
datasets = ['mnist', 'cifar10']
baseline = {}
for ds in datasets:
    for mt in models:
        accs = run_experiment(ds, mt, epochs=5, batch_size=32, lr=1e-3)
        baseline[(ds, mt)] = accs.mean()
# 绘制柱状图
labels = ['MNIST-BP', 'MNIST-CNN', 'CIFAR-BP', 'CIFAR-CNN']
values = [baseline[(ds,mt)] for ds,mt in [('mnist','bp'),('mnist','cnn'),('cifar10','bp'),('cifar10','cnn')]]
plt.figure(figsize=(8,5))
plt.bar(labels, values)
plt.ylabel('Mean Test Accuracy')
plt.title('BP vs. CNN on MNIST & CIFAR-10')
plt.ylim(0,1)
plt.savefig('compare_baseline.png')
plt.close()

Epoch 1/5 - Train loss: 0.4962, Train acc: 0.8414 | Val loss: 0.1991, Val acc: 0.9474
Epoch 2/5 - Train loss: 0.1940, Train acc: 0.9489 | Val loss: 0.1801, Val acc: 0.9551
Epoch 3/5 - Train loss: 0.1511, Train acc: 0.9626 | Val loss: 0.1930, Val acc: 0.9545
Epoch 4/5 - Train loss: 0.1328, Train acc: 0.9670 | Val loss: 0.2934, Val acc: 0.9376
Epoch 5/5 - Train loss: 0.1110, Train acc: 0.9719 | Val loss: 0.1190, Val acc: 0.9710
Test loss: 0.1231, Test accuracy: 0.9707
Epoch 1/5 - Train loss: 0.4749, Train acc: 0.8445 | Val loss: 0.2030, Val acc: 0.9429
Epoch 2/5 - Train loss: 0.1704, Train acc: 0.9561 | Val loss: 0.1696, Val acc: 0.9541
Epoch 3/5 - Train loss: 0.1217, Train acc: 0.9681 | Val loss: 0.1644, Val acc: 0.9621
Epoch 4/5 - Train loss: 0.0986, Train acc: 0.9746 | Val loss: 0.1558, Val acc: 0.9613
Epoch 5/5 - Train loss: 0.0813, Train acc: 0.9794 | Val loss: 0.1237, Val acc: 0.9708
Test loss: 0.1115, Test accuracy: 0.9726
Epoch 1/5 - Train loss: 0.4989, Train acc: 0.8427 | Val lo

In [4]:
lrs = [1e-4, 1e-3, 5e-3, 1e-2]
sweep_lr = { lr: {} for lr in lrs }
for lr in lrs:
    for mt in models:
        acc = run_experiment('mnist', mt, epochs=5, batch_size=32, lr=lr).mean()
        sweep_lr[lr][mt] = acc
plt.figure(figsize=(8,5))
for mt, color in zip(models, ['#4C72B0','#C44E52']):
    accs = [sweep_lr[lr][mt] for lr in lrs]
    plt.plot(lrs, accs, marker='o', label=mt.upper(), linewidth=2)
plt.xscale('log')
plt.xlabel('Learning Rate (log scale)', fontsize=12)
plt.ylabel('MNIST Mean Test Accuracy', fontsize=12)
plt.title('4.2 Effect of Learning Rate on BP & CNN', fontsize=14)
plt.grid(True, which="both", linestyle='--', alpha=0.7)
plt.legend(fontsize=10)
plt.tight_layout()
plt.savefig('lr_sensitivity.png', dpi=300)
plt.close()

Epoch 1/5 - Train loss: 0.8904, Train acc: 0.6966 | Val loss: 0.4355, Val acc: 0.8762
Epoch 2/5 - Train loss: 0.3690, Train acc: 0.8925 | Val loss: 0.3129, Val acc: 0.9111
Epoch 3/5 - Train loss: 0.2645, Train acc: 0.9243 | Val loss: 0.2491, Val acc: 0.9287
Epoch 4/5 - Train loss: 0.2032, Train acc: 0.9425 | Val loss: 0.2109, Val acc: 0.9407
Epoch 5/5 - Train loss: 0.1589, Train acc: 0.9554 | Val loss: 0.1800, Val acc: 0.9493
Test loss: 0.1736, Test accuracy: 0.9497
Epoch 1/5 - Train loss: 1.0631, Train acc: 0.6367 | Val loss: 0.5220, Val acc: 0.8538
Epoch 2/5 - Train loss: 0.3948, Train acc: 0.8906 | Val loss: 0.3235, Val acc: 0.9118
Epoch 3/5 - Train loss: 0.2623, Train acc: 0.9286 | Val loss: 0.2415, Val acc: 0.9339
Epoch 4/5 - Train loss: 0.2015, Train acc: 0.9440 | Val loss: 0.1977, Val acc: 0.9445
Epoch 5/5 - Train loss: 0.1640, Train acc: 0.9543 | Val loss: 0.1838, Val acc: 0.9483
Test loss: 0.1812, Test accuracy: 0.9504
Epoch 1/5 - Train loss: 1.0032, Train acc: 0.6671 | Val lo

In [ ]:
# 4.3 激活函数对比实验
activations = ['relu', 'leakyrelu', 'sigmoid']
sweep_act = { act: {} for act in activations }
for act in activations:
    for mt in models:
        acc = run_experiment('cifar10', mt, epochs=5, batch_size=32, lr=1e-3, activation=act).mean()
        sweep_act[act][mt] = acc
plt.figure(figsize=(8,5))
for mt, marker, color in zip(models, ['o','s'], ['#4C72B0','#C44E52']):
    accs = [sweep_act[act][mt] for act in activations]
    plt.plot(activations, accs, marker=marker, label=mt.upper(), linewidth=2)
plt.xlabel('Activation Function', fontsize=12)
plt.ylabel('MNIST Mean Test Accuracy', fontsize=12)
plt.title('4.4 Effect of Activation Function on BP & CNN', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(fontsize=10)
for i, act in enumerate(activations):
    for mt in models:
        plt.text(i, sweep_act[act][mt]+0.005, f"{sweep_act[act][mt]:.2%}", ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.savefig('activation_sensitivity.png', dpi=300)
plt.close()

Epoch 1/5 - Train loss: 0.5213, Train acc: 0.8289 | Val loss: 0.2428, Val acc: 0.9363
Epoch 2/5 - Train loss: 0.1861, Train acc: 0.9510 | Val loss: 0.1817, Val acc: 0.9518
Epoch 3/5 - Train loss: 0.1363, Train acc: 0.9642 | Val loss: 0.1395, Val acc: 0.9634
Epoch 4/5 - Train loss: 0.1073, Train acc: 0.9721 | Val loss: 0.1315, Val acc: 0.9670
Epoch 5/5 - Train loss: 0.0969, Train acc: 0.9755 | Val loss: 0.1245, Val acc: 0.9697
Test loss: 0.1253, Test accuracy: 0.9706
Epoch 1/5 - Train loss: 0.6383, Train acc: 0.7676 | Val loss: 0.2498, Val acc: 0.9317
Epoch 2/5 - Train loss: 0.1859, Train acc: 0.9516 | Val loss: 0.1897, Val acc: 0.9498
Epoch 3/5 - Train loss: 0.1284, Train acc: 0.9671 | Val loss: 0.1289, Val acc: 0.9664
Epoch 4/5 - Train loss: 0.0999, Train acc: 0.9740 | Val loss: 0.1402, Val acc: 0.9655
Epoch 5/5 - Train loss: 0.0832, Train acc: 0.9786 | Val loss: 0.1266, Val acc: 0.9705
Test loss: 0.1136, Test accuracy: 0.9721
Epoch 1/5 - Train loss: 0.4972, Train acc: 0.8404 | Val lo

In [ ]:
from wandb import sweep

# 4.4 结果分析
print("4.1 对比实验结果：")
for (ds, mt), acc in baseline.items():
    print(f"- {ds.upper()} + {mt.upper()}: {acc:.4%}")
print("\n4.2 学习率敏感性：")
for lr, res in sweep.items():
    for mt, acc in res.items():
        print(f"- LR={lr:.1e}, {mt.upper()}: {acc:.4%}")

print("\n分析示例：")
print("1. CNN 显著优于 BP，表明局部卷积结构对图像任务更有效。")
print("2. BP 对学习率更敏感，过大或过小均影响性能；CNN 对 lr 稳定性更好。")
print("3. 小方差说明 5-折结果稳定，若不稳定应加入正则化或数据增强。")